In [1]:
import os
import torch
import random
import evaluate
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import random

import torch
from torch.utils.data import DataLoader


from sklearn.model_selection import train_test_split

from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from dataset import BERTDataset
import evaluate
from datasets import load_dataset, Dataset
from sklearn.metrics import f1_score


ImportError: cannot import name 'BERTDataset' from 'dataset' (/opt/conda/lib/python3.10/site-packages/dataset/__init__.py)

In [4]:


# ================================================
print("데이터셋 로드 및 설정")
# ================================================

# 랜덤 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)


# 모델과 토크나이저 로드
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

df_train = pd.read_csv("/data/ephemeral/home/data/train.csv")
#df_valid = pd.read_csv("/data/ephemeral/home/data/valid_output.csv")


# 데이터셋에 인덱스 컬럼 추가
df_train = df_train.reset_index().rename(columns={'index': 'idx'})
#df_valid = df_valid.reset_index().rename(columns={'index': 'idx'})


데이터셋 로드 및 설정


In [5]:
df_train_pc = df_train.copy()

In [6]:
import string
# 구두점 문자 정의
punctuation = string.punctuation

# 구두점 개수 계산 함수
def count_punctuation(text):
    return sum(1 for char in text if char in punctuation)

In [7]:
# DataFrame의 텍스트 열에 함수 적용 (예: 'text' 열이 텍스트 데이터를 포함한다고 가정)
df_train_pc['punctuation_count'] = df_train_pc['text'].apply(count_punctuation)

# 전체 문자 수 계산
df_train_pc['total_chars'] = df_train_pc['text'].str.len()

In [8]:
# 구두점 비율 계산
total_punctuation = df_train_pc['punctuation_count'].sum()
total_chars = df_train_pc['total_chars'].sum()

punctuation_ratio = total_punctuation / total_chars
print(f"전체 구두점 비율: {punctuation_ratio:.2%}")

전체 구두점 비율: 7.64%


In [9]:
df_filtered = df_train_pc[df_train_pc['punctuation_count'] <= 3]

df_filtered

,idx,ID,text,target,punctuation_count,total_chars
0,0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,3,32
2,2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,3,22
3,3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,0,29
4,4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,2,28
5,5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,0,30
...,...,...,...,...,...,...
2795,2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,0,33
2796,2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,0,27
2797,2797,ynat-v1_train_02797,텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보,4,2,32
2798,2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,0,31


In [10]:
import pandas as pd
from collections import Counter
import re

# 한국어 불용어 리스트 (예시, 필요에 따라 확장 가능)
korean_stopwords = set(['은', '는', '이', '가', '을', '를', '의', '에', '에서', '로', '으로', '과', '와', '도', '만', '에게', '께', '한테', '보다', '라고', '이라고', '으로서', '같이', '처럼', '만큼'])

# 정규표현식 패턴 정의
korean_pattern = re.compile('[가-힣]{2,}')

In [11]:
def process_text(text):
    # 영어와 특수문자 제거, 소문자 변환
    text = re.sub(r'[a-zA-Z0-9\W]+', ' ', text.lower())
    
    # 한글 단어 추출 (2글자 이상)
    words = korean_pattern.findall(text)
    
    # 불용어 제거
    words = [word for word in words if word not in korean_stopwords]
    
    return words

# 모든 텍스트에서 단어 추출 및 빈도 계산
all_words = []
for text in df_filtered['text']:
    all_words.extend(process_text(text))

word_counts = Counter(all_words)

# 상위 50개 단어 추출
top_50_words = word_counts.most_common(50)

# 결과 출력
for word, count in top_50_words:
    print(f"{word}: {count}")

대통령: 50
게시판: 34
출시: 27
이란: 25
삼성: 23
한국: 20
감독: 20
개발: 19
네이버: 18
분기: 17
억원: 17
트럼프: 17
만에: 16
개최: 16
개막: 15
갤럭시: 15
신간: 15
전국: 15
추진: 15
증가: 14
류현진: 14
정부: 14
내년: 14
경기: 13
연속: 13
공개: 13
프로농구: 13
코스피: 12
올해: 12
김정은: 12
애플: 12
스마트폰: 11
여행: 11
서비스: 11
민주: 11
그래픽: 11
최대: 11
오늘: 11
지원: 11
아이폰: 10
성공: 10
아시안게임: 10
다시: 10
국내: 10
종합: 10
홍콩: 10
세계: 10
선정: 10
대표: 10
하락: 10


In [12]:
df_test = df_filtered.head(100)

In [13]:
df_test

,idx,ID,text,target,punctuation_count,total_chars
0,0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,3,32
2,2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,3,22
3,3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,0,29
4,4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,2,28
5,5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,0,30
...,...,...,...,...,...,...
139,139,ynat-v1_train_00139,떼었S 붙였다 액세서리Y 스마C폰 업U레이드,4,0,24
140,140,ynat-v1_train_00140,무등산에서 광주지역 올해 첫 얼음 관찰,5,0,21
142,142,ynat-v1_train_00142,朴대통령 與 TK 국회의원과 110분 면담…사드 민심 청취,0,0,32
143,143,ynat-v1_train_00143,지소미아 종료까지 8일…美 압박 속 고민 깊어지는 文대통령,4,0,32


In [12]:
'''
MLP-KTLim/llama-3-Korean-Bllossom-8B
250GB의 한국어 데이터로 사전학습
LogicKor 벤치마크에서 10B 이하 한국어 모델 중 최고 성능 기록
3만 개 이상의 한국어 어휘 확장
한국어-영어 이중 언어 모델

PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct
한국어 자연어 처리 작업에 특화된 모델
다양한 복잡한 지시사항을 처리할 수 있도록 설계됨

saltlux/Ko-Llama3-Luxia-8B
100GB 이상의 한국어 데이터로 사전학습
Llama-3 토크나이저를 한국어로 확장 (17,536개의 한국어 토큰 추가)

beomi/Llama-3-Open-Ko-8B
60GB 이상의 한국어 텍스트로 사전학습
17.7B+ 토큰으로 학습

'''

'\nMLP-KTLim/llama-3-Korean-Bllossom-8B\n250GB의 한국어 데이터로 사전학습\nLogicKor 벤치마크에서 10B 이하 한국어 모델 중 최고 성능 기록\n3만 개 이상의 한국어 어휘 확장\n한국어-영어 이중 언어 모델\n\nPathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct\n한국어 자연어 처리 작업에 특화된 모델\n다양한 복잡한 지시사항을 처리할 수 있도록 설계됨\n\nsaltlux/Ko-Llama3-Luxia-8B\n100GB 이상의 한국어 데이터로 사전학습\nLlama-3 토크나이저를 한국어로 확장 (17,536개의 한국어 토큰 추가)\n\nbeomi/Llama-3-Open-Ko-8B\n60GB 이상의 한국어 텍스트로 사전학습\n17.7B+ 토큰으로 학습\n\n'

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [69]:
model_id = "beomi/Llama-3-Open-Ko-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [74]:
words = ["대통령", "게시판", "출시", "이란", "삼성", "한국", "감독", "개발", "네이버", 
         "분기", "억원", "트럼프", "만에", "개최", "개막", "갤럭시", "신간", "전국", "추진", "증가", "류현진", "정부", "내년", 
         "경기", "연속", "공개", "프로농구", "코스피", "올해", "김정은", "애플", "스마트폰", "여행", "서비스", "민주", "그래픽", "최대",
           "오늘", "지원", "아이폰", "성공", "아시안게임", "다시", "국내", "종합", "홍콩", "세계", "선정", "대표", "하락"]

In [75]:
prompt = f"""다음 단어 목록을 사용하여 7개의 뉴스 카테고리로 분류해주세요:

{', '.join(words)}

각 카테고리를 다음 형식으로 제시해주세요:

Keyword: [카테고리 이름]
Potential Categories: [관련 단어들을 쉼표로 구분하여 나열]

예시:
Keyword: 정치/외교
Potential Categories: 대통령, 트럼프, 김정은, 정부, 민주, 이란, 홍콩

위 예시와 같은 형식으로 7개의 카테고리를 모두 작성해주세요. 각 카테고리는 고유해야 하며, 단어들이 중복되지 않도록 해주세요."""

In [77]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000, do_sample=True, temperature=0.3)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [78]:
print(response)

다음 단어 목록을 사용하여 7개의 뉴스 카테고리로 분류해주세요:

대통령, 게시판, 출시, 이란, 삼성, 한국, 감독, 개발, 네이버, 분기, 억원, 트럼프, 만에, 개최, 개막, 갤럭시, 신간, 전국, 추진, 증가, 류현진, 정부, 내년, 경기, 연속, 공개, 프로농구, 코스피, 올해, 김정은, 애플, 스마트폰, 여행, 서비스, 민주, 그래픽, 최대, 오늘, 지원, 아이폰, 성공, 아시안게임, 다시, 국내, 종합, 홍콩, 세계, 선정, 대표, 하락

각 카테고리를 다음 형식으로 제시해주세요:

Keyword: [카테고리 이름]
Potential Categories: [관련 단어들을 쉼표로 구분하여 나열]

예시:
Keyword: 정치/외교
Potential Categories: 대통령, 트럼프, 김정은, 정부, 민주, 이란, 홍콩

위 예시와 같은 형식으로 7개의 카테고리를 모두 작성해주세요. 각 카테고리는 고유해야 하며, 단어들이 중복되지 않도록 해주세요.이러한 결과는 이 연구에서 사용된 5가지 항목 중 하나인 '일상생활에서 다른 사람을 도와주는 것'이 사회적 자본의 구성 요소인 사회적 지원과 밀접한 관련이 있음을 보여준다.
These results show that 'helping others in daily life', one of the five items used in this study, is closely related to social support, a component of social capital.
These results show that 'helping others in daily life', one of the five items utilized in this study, is closely related to social support, a component of social capital.
 2월 20일(목)부터 3월 1일(일)까지 10일간 온라인으로 개최되는 이번 전시회는 2월 20일(목) 개막식을 시작으로 3월 1일(일)까지

In [1]:
del tokenizer
del model

NameError: name 'tokenizer' is not defined

In [ ]:
del tokenizer
del model

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
import gc
gc.collect()

0